In [1]:
!pip uninstall -y keras tensorflow transformers
!pip install tensorflow==2.12.0
!pip install keras==2.12.0
!pip install transformers
!pip install tf-keras

Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: transformers 4.47.1
Uninstalling transformers-4.47.1:
  Successfully uninstalled transformers-4.47.1
  Obtaining dependency information for tensorflow==2.12.0 from https://files.pythonhosted.org/packages/b0/a1/0f6dbe2526803c1077bc2a2afefbc29d4be92bb4dfa11e96d45cdc098e36/tensorflow-2.12.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached tensorflow-2.12.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Obtaining dependency information for keras<2.13,>=2.12.0 from https://files.pythonhosted.org/packages/d5/80/34e55d7e3ed9cf18020929460f969de1bf82cf2f509c639b358ae2b25618/keras-2.12.0-py2.py3-none-any.whl.metadata
  Using cached keras-2.12.0-py2.py3-none-any.whl.metadata (1.4 k

In [1]:
import warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')
import os
import re
import csv
import datetime
# from transformers import pipeline
# from tokenizers.decoders import WordPiece
import re
import os
import requests
from bs4 import BeautifulSoup


In [2]:
def get_urls():
    """
    get all the urls of the text files from the oknesset website.
    """
    url = 'https://production.oknesset.org/pipelines/data/committees/meeting_protocols_text/files/'
    folder_urls = [url]
    urls = []

    while(len(folder_urls) > 0):
        curr_url = folder_urls.pop(0)
        response = requests.get(curr_url, allow_redirects=False)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            all_links = soup.find_all('a')
            file_names = [link.text for link in all_links if link.text]
            file_names = [file_name for file_name in file_names if file_name != '..']
            for file_name in file_names:
                if file_name.endswith('/') and file_name != '../':
                    folder_urls.append(curr_url + file_name)
                elif file_name.endswith('.txt'):
                    urls.append(curr_url + file_name)
        else:
            print(f"Failed to retrieve content from {url}. Status code: {response.status_code}")
    return urls

def filter_by_call_to_order(text):
    """
    check if the text contains a call to order.
    """
    calls_to_order = offensive_words =[
        "אהצבועה", "אובססיבית", "איכס", "אינעל","שמוק","דביל","בולבול","קפוץ לי", "אנטיפתי", "אסתור", "ב\"ביצים", "בביצים", "בגועליציה", "בגזענותו",
        "בוזין", "בזוי", "בזויים", "בחרא", "ביריונים", "בנזונה", "בקקה", "בשמוק", "בתחת", "בתחתונים", "גונב", "גונבי",
        "גנב", "גנבים", "גנבת", "דימגוג", "דפוק", "דפוקים", "דרקים", "הארכיטיפ", "הבוגדים", "הזבל", "הזבלל", "הזדוניים",
        "הזוי", "הזויה", "הזויים", "החאפרים", "החארות", "החולנית", "החולרה", "החוצפן", "החמור", "יחמור", "החפרן",
        "היללן", "המבהיל", "המבחילה", "המושרצים", "המזורגג", "המחורבן", "המחורבנות", "המחורבנים", "המחורבנת", "המטומטם",
        "המטומטמת", "המנוולת", "המניאקים", "המנייאק", "המסריח", "המסריחים", "המפלצת", "המתוסבכתנכדתו", "הסטן",
        "הסרחוני", "הפאקינג", "הפושע", "הפושעים", "הפראייר", "הציבוז'י", "הקקה", "הרשע", "הרשעים", "הרשעית", "השמוק",
        "השמוקים", "השקרן", "השקרנית", "השרלטנות", "התחת", "וגנב", "ודרעק", "והנבער", "והנצלנית", "והשפל", "וחלאות",
        "וחרא", "וכוסאמק", "ולברבר", "ולעלובי", "ומנוול", "ומנוולים", "ומסריח", "וסורר", "וסטנים", "וסמרטוטית",
        "וערמומית", "ופחדן", "ופטפטני", "ופסיכופטים", "וצבוע", "וצבועה", "וציפורע", "וקשקשן","קשקשן" "ורמאי", "ורמאים",
        "ושקרן", "ושקרנים", "ושקרנית", "זבל", "זבלה", "זבלים", "זומבים", "חאלס", "חאפר", "חאפרים", "חאפרית",
        "חובבן", "חולני", "חזיר", "חירבנה", "חמוריםםם", "חרא", "חראא", "חראאא", "טינופת", "טמטום", "יאוכלי", "יבן",
        "יוק", "יזבל", "יזבלה", "יזבל", "יחתיכת", "יטמבל", "יליצן", "ימגעיל", "ימיץ", "ימניאק", "ימניייק", "ימנייק",
        "ימניק", "ימעפן", "ינעל", "כאןצבועה", "ככלבלב", "יכלב", "כנפיחה", "כפודל", "כשחלאה", "לאזעזל", "להזויים",
        "להשתין", "לזרגג", "לחאפר", "לחרא", "לחרבן", "לךגזען", "למותי", "לעזאזאל", "לעזזאל", "לקק", "לשקרנים",
        "מאוסה", "מאפן", "מבהיל", "מבהילה", "מבהילים", "מבחילה", "מגעיל", "מגעילה", "מגעילים", "מהאדיוט", "מזוויעה",
        "מזוויעים", "מזורגג", "מזנים", "מחורבן", "מחורבנות", "מחורבנת", "מטונף", "מלהתקרצץ", "מלשין", "מלשינה",
        "ממזרים", "מנובל", "מנובלים", "מניאק", "מנייאק", "מנייק", "מניפולטורית", "מניפולטיבית", "מניק", "מנניאיק",
        "מסקן", "מסקנים", "מסריח", "מסריחה", "מסריחות", "מסריחים", "מעפן", "מעפנהה", "מפגר", "מפגרים", "מפגרת",
        "מפחידה", "משקר", "משקרים", "משתין", "משתינה", "משתינות", "משתינים", "נבזי", "נבזים", "נוכל", "נוכלות",
        "נוכלים", "נוכלת", "נחש", "ניבזה", "נצלנים", "סאדיסטי", "סטן", "סטנים", "סטרפלצט", "סמרטוט", "עוכר",
        "עוכרי", "עוכרת", "פָּתט", "פאקינג", "פברק", "פברקו", "פושע", "פושעת", "יפח", "פחדנים", "פחזבל", "פיפי",
        "פלוץ", "פתטי", "פתי", "צבוע", "צבועה", "צבועים", "קברן", "קברניות", "קברנים", "קברנית", "קיבינמט",
        "קקאיות", "קקה", "קקות", "קקי", "קרימינל", "קרימינלית", "רבאק", "רמאי", "רמאים", "רמאית", "רשלן",
        "שאובססיבית", "שהמפלצת", "שודד", "שונא", "שונאת", "שטחית", "שלוזרים", "שמוק", "שמלאני", "שמלאנים",
        "שמלנים", "שפיצרקה", "שקלוזה", "שקרן", "שקרני", "שקרניות", "שקרנים", "שקרנית", "שרלטן", "שרלטנים",
        "שרלטנית", "שתחנק", "שתמות", "שתמותי", "שתסתמי", "שתשרפי", "תחמן", "תחמניות", "תחמנים", "תחמנית",
        "תיסתום", "תישארמה", "תמותי", "תנענע", "תסתום", "תסתמו", "תסתמי", "תעופי", "תעפו", "תשרף", "תשתכשכי"
        , "- - -",  "- -" , 'אני קורא אותך לסדר', 'אני קוראת אותך לסדר', 'זאת אזהרה אחרונה',
       "קריאה לסדר", "קריאת ראשונה לסדר", "אני מזהיר אותך", "קורא אותך", "קוראת אותך", "קורא לך לסדר", 
    "קוראת לך לסדר"
]
    for call in calls_to_order:
        if call in text:
            return True
    return False

def get_text(url, dir='text'):
    """
    get the text from the url and save it in a file in the dir. Only if the text contains a call to order.
    """
    response = requests.get(url)
    if response.status_code == 200:
        response.encoding = 'utf-8-sig'
        if not os.path.exists(dir):
            os.makedirs(dir)
        text = response.text
        text = text.replace('”', '"').replace('“', '"').replace('״', '"')
        if filter_by_call_to_order(text):
            text = re.split('\n', text)
            with open(f'{dir}/{url.split("/")[-1]}', 'x', encoding='utf-8-sig') as file:
                file.write('\n'.join(text))
        else:
            print(f"Not found call to order in {url}.")

    else:
        print(f"Failed to retrieve content from {url}. Status code: {response.status_code}")
        return None  

In [3]:
def main():
    """
    Main function to retrieve, filter, and save meeting protocol texts.
    """
    # Step 1: Get all URLs
    print("Fetching URLs from the Oknesset website...")
    urls = get_urls()
    print(f"Total URLs fetched: {len(urls)}")

    # Step 2: Filter and save texts
    for url in urls:
        try:
            print(f"Processing: {url}")
            get_text(url)
        except Exception as e:
            print(f"Error processing {url}: {e}")

    print("Completed processing all URLs.")

# Run the main function
if __name__ == "__main__":
    main()


Fetching URLs from the Oknesset website...
Failed to retrieve content from https://production.oknesset.org/pipelines/data/committees/meeting_protocols_text/files/. Status code: 504
Total URLs fetched: 33720
Processing: https://production.oknesset.org/pipelines/data/committees/meeting_protocols_text/files/1/0/100017.txt
Error processing https://production.oknesset.org/pipelines/data/committees/meeting_protocols_text/files/1/0/100017.txt: [Errno 17] File exists: 'text/100017.txt'
Processing: https://production.oknesset.org/pipelines/data/committees/meeting_protocols_text/files/1/0/100018.txt
Error processing https://production.oknesset.org/pipelines/data/committees/meeting_protocols_text/files/1/0/100018.txt: [Errno 17] File exists: 'text/100018.txt'
Processing: https://production.oknesset.org/pipelines/data/committees/meeting_protocols_text/files/1/0/100019.txt
Error processing https://production.oknesset.org/pipelines/data/committees/meeting_protocols_text/files/1/0/100019.txt: [Errno 